In [ ]:
'''
Physics-Informed Convolutional model to predict
the diagonal Reynolds stress tensor terms, which
can later be used to calculate Turbulent Pressure.

The example data is a sampled version of the 3D CCSN
data used in P.I.Karpov, arXiv:2205.08663

The algorythm is best suited to be trained on GPU/s,
but for compatibility reasons the 'device' is set to 'cpu'
in this example. Please change it if you have CUDA enabled system

Note: due to a custom loss implementation, Catalyst is NOT used,
hence logging is handled manually for this example!

-pikarpov
'''
%matplotlib inline

import os
import sys

sys.path.append("../../")

from sapsan.lib.backends import MLflowBackend
from sapsan.lib.data import HDF5Dataset, EquidistantSampling, flatten
from sapsan.lib.estimator.pimlturb.pimlturb_diagonal_estimator import PIMLTurb, PIMLTurbConfig
from sapsan import Train, Evaluate, model_graph
from sapsan.utils.plot import log_plot

In [ ]:
#--- Experiment tracking backend ---
#MLflow - the server will be launched automatically
#in case it won't, type in cmd: mlflow ui --port=9000
#uncomment tracking_backend to use mlflow

experiment_name = "PIMLTurb experiment"
#tracking_backend = MLflowBackend(experiment_name, host="localhost", port=9000)

In [ ]:
#--- Data setup ---
#In the intereset of loading and training multiple timesteps
#one can specify which checkpoints to use and where
#they appear in the path via syntax: {checkpoint:format}
#
#Next, you need to specify which features to load; let's assume 
#        path = "{feature}.h5"
#
# 1) If in different files, then specify features directly;
#    The default HDF5 label will be the last label in the file
#    Ex: features = ['velocity', 'denisty', 'pressure']
# 2) If in the same, then duplicate the name in features
#    and specify which labels to pull
#    Ex: features = ["data", "data", "data"]
#        feature_labels = ['velocity', 'density', 'pressure']

sigma = 9
base = "data/ccsn-mri-sims/"
path = base + "{feature}_dim17_s%d_it{checkpoint:1.0f}.h5"%(sigma)

features = ['vel', 'Bvec', 'dvel', 'dBvec']
target = ['tnvel']

#tensor component to train against, with the following layout:
# 0  1  2
# 3  4  5 
# 6  7  8

tn_comp = 0

#checkpoints to train on
checkpoints = [40,60,80,100,120,140,160,180,200,220]

#Dimensionality of your data in format [D,H,W]
INPUT_SIZE = [17,17,17]

In [ ]:
#Load the data
#train_fraction = len(checkpoints)-1) leaves 1 batch for validation

data_loader = HDF5Dataset(path=path,
                          features=features,
                          target=target,
                          checkpoints=checkpoints,
                          input_size=INPUT_SIZE,
                          train_fraction = len(checkpoints)-1)
x, y = data_loader.load_numpy()

y = y[:, tn_comp:tn_comp+1]

loaders = data_loader.convert_to_torch([x, y])

In [ ]:
#Machine Learning model to use

#Configuration of the model parameters:
#    n_epochs = number of epochs (need ~1000 for sensible results)
#    patience = number of epochs to run beyond convergence
#    min_delta = loss based convergence cut-off
#    lr = learning rate
#    min_lr = minimum learning rate
#    device = device to train the model on: ['cpu', 'cuda', 'cuda:0', ...]
#    activ = activation function
#    loss = loss function
#    ks_stop = at wait value of the KS (Kolmogorov-Smirnov stat) to stop training
#              Note: NOT the total loss value, but only the KS component!
#    scalel1 = by what amount to scale the initial L1 loss, so that it is
#              reduced first, with KS loss dominating later in the training
#    betal1 = beta for the Smooth L1 (cutoff point for smooth portion)
#    sigma = sigma for the Gaussian kernel for the filtering step of each epoch

estimator = PIMLTurb(
        config = PIMLTurbConfig(n_epochs=10, patience=10, 
                                min_delta=1e-7, lr=1e-4, min_lr=1e-4*1e-5,
                                device='cpu'),
        loaders = loaders,        
        activ="Tanhshrink", loss="SmoothL1_KSLoss",
        ks_stop = 0.04,
        scalel1 = 1e6,
        betal1 = 1,
        sigma = 1)

In [ ]:
#Given the lower resolution, the published results cannot be achieved.
#However, after ~1000 epochs, the predictions get quite good,
#showing the effect of the custom loss.

#Set the experiment
training_experiment = Train(model = estimator,
                            #backend=tracking_backend, #uncomment to use mlflow                            
                            data_parameters = data_loader)

#Train the model
estimator = training_experiment.run()

In [ ]:
#plotting training results
#Note: not done automatically since Catalyst is NOT used
log = log_plot(show_log=True,
               log_path = 'train_log.txt', 
               valid_log_path = 'valid_log.txt', 
               delimiter = '\t')
log_name = "runtime_log.html"
log.write_html(log_name)
#tracking_backend.log_artifact(log_name)

#--------------------------------------------------
#plotting individual evolutions of L1 and KS losses
#Note: the individual losses are not logged at every epoch
losses = [["mean(L1_loss)", "mean(L1_valid)"],
          ["mean(KS_loss)", "mean(KS_valid)"]]

for idx, (train_name, valid_name) in enumerate(losses): 
    log = log_plot(show_log=True, 
                   valid_log_path = 'losses_valid_log.txt',
                   log_path = 'losses_train_log.txt',
                   delimiter = '\t',
                   train_name=train_name, valid_name=valid_name,
                   train_column=idx, valid_column=idx, epoch_column=None)
    log_name = f"{train_name}.html"
    log.write_html(log_name)    
    #tracking_backend.log_artifact(log_name) 

In [ ]:
%matplotlib inline

checkpoints_eval = [233,313,393]
for chk in checkpoints_eval:
    #Load the test data
    data_loader_eval = HDF5Dataset(path=path,
                       features=features,
                       target=target,
                       checkpoints=[chk],
                       input_size=INPUT_SIZE,
                       )
    test_x, test_y = data_loader_eval.load_numpy()
    test_y = test_y[:, tn_comp:tn_comp+1,::]
    loaders_eval = data_loader_eval.convert_to_torch([test_x, test_y])

    #Set the test experiment
    estimator.loaders = loaders_eval
    evaluation_experiment = Evaluate(model = estimator,
                                     #backend = tracking_backend,
                                     data_parameters = data_loader_eval,
                                     pdf_xlim = (-0.5e-3,1.5e-3),
                                     cdf_xlim = (-0.5e-3,1.5e-3))


    #Test the model; get the dictionary of {'predict', 'target'}
    prediction = evaluation_experiment.run()
    
    #logs extra parameters
    #run_id = evaluation_experiment.run_id
    #tracking_backend.resume(run_id = run_id)
    #tracking_backend.log_parameter("tn_comp", tn_comp)    
    #tracking_backend.end()    